In [ ]:
import cv2 #Libraries.
import numpy as np
import matplotlib.pyplot as plt
import math
import time

sigma_value_i=8  #Some values for initialization, we can change their value to apply different filter.
sigma_value_s=20
squareSize=7
gaus_list=[]    #Simply the Gaussian values list.There can be a 255-i , 255-i+1 ,255-i+2.. 255-i+255 possibilities
                #for gaussain values. I calculated all the possibilites for gaussian values so no need to do extra process for every pixel.

distance_list=[]  #Same as gauslist. There is a 7x7 square and distances between center coordinates and other coordinates always be the same.
                  #so again no need to calculate for everytime.

#Simply to optimize the code I use RAM instead of CPU.

In [ ]:
def find_gaussian_values():    #Find gaussian values and add them to a list.
    global sigma_value_i
    i=1
    while i<257:
        gaus_list.append(0)
        i+=1
    counter=0
    i=0
    while(i<255):
        x= 255-counter
        gaus_list[x-1]=((1/(2*math.pi*(sigma_value_i**2)))*math.exp(-(x ** 2)/(2*sigma_value_i**2)))
        i+=1
        counter+=1

def find_distance_values():    #Find center pixel distances from other pixels in 7x7 square and add them to a list.
    global sigma_value_s,squareSize
    halflenOfSquare=squareSize//2
    x=0
    while(x<7):
        y=0
        distance_list.append([])
        while(y<7):
            distance_list[x].append((1/(2*math.pi*(sigma_value_s ** 2)))*math.exp(- (((halflenOfSquare-x)**2 +(halflenOfSquare-y)**2))/(2*sigma_value_s**2)) )
            y+=1
        x+=1
        
find_gaussian_values()       #CALL the functions because we will need a list.
find_distance_values()


In [ ]:
def bilateral_filter(originalImg): #Take original image as a parameter.
    bileteralImage = np.zeros(originalImg.shape)   #Create a image in original image size.
    halfOfSizeSquare = squareSize//2   #We get half of the of square for visiting the pixels.
    i = 0   #There will be a 4 loop, first rows in image then their columns and visiting neighbours for a 7x7 square.
    for row in img:
        j = 0
        for pixel in row:
            image_filtered_pixel = 0 #TO calculate new pixel value we set 0 here, so in the other loop will be clear and value will be 0.
            Wp = 0  #Same as above.
            for rowOfSquare in range(0,squareSize):   
                for ColumnOfSquare in range(0,squareSize):   
                    visitForX = i - (halfOfSizeSquare - rowOfSquare)
                    visitForY = j - (halfOfSizeSquare - ColumnOfSquare)
                    if visitForX >= len(originalImg): #For edge pixels.If it is in edge or corner pixel simply get current pixel idensity of it.
                        visitForX = i
                    if visitForY >= len(originalImg[0]): #For edge pixels.If it is in edge or corner pixel simply get current pixel idensity of it.
                        visitForY = j    
                    w = distance_list[rowOfSquare][ColumnOfSquare] * gaus_list[originalImg[visitForX][visitForY] - originalImg[i][j]]  #Wikipedia formula and we use the lists in here.
                    image_filtered_pixel += originalImg[visitForX][visitForY] * w  #We visit neigbours and calculate new pixel 
                    Wp += w    
            bileteralImage[i][j] =image_filtered_pixel // Wp   # Since it will be an integer I used //.
            j+=1
       #To open loading screen enable to code below please.
       # print ("Loading bileteral filter %:",(i/len(originalImg)*100))  #Print to the screen loading percentage. If we get rid of this print command our process will be faster.
        i+=1
    return bileteralImage  #return the new image.

In [ ]:
img = cv2.imread("in_img.jpg",0)
start = time.time()
myBileteralImg = bilateral_filter(img)
cv2.imwrite("filtered_image_own.png", myBileteralImg)
print("Modification takes:",(time.time()-start)*1000.0,"ms")

image_OpenCV = cv2.bilateralFilter(img, 7, 8, 20)
cv2.imwrite("filtered_image_OpenCV.png", img)

#### I started with sigma_value_i,sigma_value_s= 2,2: applied the filter. Seems like our image changed very little, almost nothing.

#### I tried sigma_value_i,sigma_value_s= 20,2 Now the image lost deatils in the roof. I think the noises are gone and  image brightness  is increased. 

#### I tried sigma_value_i,sigma_value_s= 200,2 to see what will happen to an image. Image changed like there is man whose eyes are short-sighted (miyop) and can't see the far away. Blur level of image is increased very much.

#### I have learned how sigma_value_i is worked.

#### I tried sigma_value_i,sigma_value_s= 20,10 and sigma_value_i,sigma_value_s= 20,2.

#### I noticed that sgma_value_s is increase in the image at the roof darker side will be darker.. And black lines in the image appears.I think this is for the edges and lines but for the larger features. I mean not to lose edges but become clear
#### If we mix both sigmas, to blur enoughly picked sigma_value_i=8 and for the lines and sigma_value_s= 20 for the lines and edges, not to lose details. I just tried many options with playing both sigmas

#### FINAL: Because sigma_value_i values changes, our image changes a lot.. So I think "8" value is enough. It blurs the image. For sigma_value_s= 20 is good for it and fixed the edges. I noticed that sigma_value_s does not make the image huge different. May be because of 7x7 size. Because every distance from pixels is fixed and not many chocies because of this is not to lose details in the edges and roof-top side line. In the other hand in "gaus_list" we have more value to implement and many options 
#### Using the bileteral rule of "Bilateral filter was introduced to smooth images while preserving edges." simply I try it to fix sigmas to this rule
